In [253]:
import pandas as pd

In [254]:
df = pd.concat([pd.read_csv(f'../usdvnd/seperated/usdvnd{year}.csv') for year in range(1994, 2026)])
df.sort_values(by='Ngày', key=lambda x: pd.to_datetime(x, format='%d/%m/%Y'), inplace=True)

In [255]:
# drop most missing values column
df.drop(columns=['KL'], inplace=True)

In [256]:
# delete special characters
df['% Thay đổi'] = df['% Thay đổi'].str.replace('%', '')
for col in df.columns:
    df[col] = df[col].str.replace(',', '')
df.head()

,Ngày,Lần cuối,Mở,Cao,Thấp,% Thay đổi
105,05/08/1994,10981.0,10981.0,10981.0,10981.0,0.00
104,08/08/1994,10981.0,10981.0,10981.0,10981.0,0.00
103,09/08/1994,10981.0,10981.0,10981.0,10981.0,0.00
102,10/08/1994,10981.0,10981.0,10981.0,10981.0,0.00
101,11/08/1994,10981.0,10981.0,10981.0,10981.0,0.00


In [257]:
# Convert date column to datetime and set as index
df['Ngày'] = pd.to_datetime(df['Ngày'], format='%d/%m/%Y')
df = df.set_index('Ngày')

# Create complete date range and reindex to fill missing dates
all_days = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')
df = df.reindex(all_days)

# Forward fill numerical columns, fill '% Thay đổi' with 0
num_cols = df.columns.drop('% Thay đổi')
df[num_cols] = df[num_cols].ffill()
df['% Thay đổi'] = df['% Thay đổi'].fillna(0)
df


,Lần cuối,Mở,Cao,Thấp,% Thay đổi
1994-08-05,10981.0,10981.0,10981.0,10981.0,0.00
1994-08-06,10981.0,10981.0,10981.0,10981.0,0
1994-08-07,10981.0,10981.0,10981.0,10981.0,0
1994-08-08,10981.0,10981.0,10981.0,10981.0,0.00
1994-08-09,10981.0,10981.0,10981.0,10981.0,0.00
...,...,...,...,...,...
2025-11-03,26312.0,26306.0,26326.5,26275.0,-0.01
2025-11-04,26320.0,26316.0,26341.0,26302.5,0.03
2025-11-05,26322.5,26319.5,26345.0,26302.5,0.01
2025-11-06,26315.0,26323.5,26338.0,26298.0,-0.03


In [258]:
# turn to numerical type
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 11418 entries, 1994-08-05 to 2025-11-07
Freq: D
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Lần cuối    11418 non-null  float64
 1   Mở          11418 non-null  float64
 2   Cao         11418 non-null  float64
 3   Thấp        11418 non-null  float64
 4   % Thay đổi  11418 non-null  float64
dtypes: float64(5)
memory usage: 535.2 KB


In [259]:
outliers = df[abs(df['% Thay đổi']) > 5]

# print outliers and its surrounding rows
print("Outliers:")
print(outliers)
print("\n" + "="*80 + "\n")

# For each outlier, show 1 day before and 1 day after
for outlier_date in outliers.index:
    start_date = outlier_date - pd.Timedelta(days=1)
    end_date = outlier_date + pd.Timedelta(days=1)
    print(f"Outlier date: {outlier_date.strftime('%Y-%m-%d')}")
    print(df.loc[start_date:end_date])
    print("\n" + "-"*80 + "\n")

Outliers:
            Lần cuối       Mở      Cao     Thấp  % Thay đổi
1998-02-16   12930.0  12930.0  12930.0  12930.0        5.22
1998-08-07   13895.0  13895.0  13895.0  13895.0        7.00
2011-02-11   20700.0  19495.0  20350.0  19490.0        6.26


Outlier date: 1998-02-16
            Lần cuối       Mở      Cao     Thấp  % Thay đổi
1998-02-15   12289.0  12289.0  12289.0  12289.0        0.00
1998-02-16   12930.0  12930.0  12930.0  12930.0        5.22
1998-02-17   12957.0  12957.0  12957.0  12957.0        0.21

--------------------------------------------------------------------------------

Outlier date: 1998-08-07
            Lần cuối       Mở      Cao     Thấp  % Thay đổi
1998-08-06   12986.0  12986.0  12986.0  12986.0       -0.05
1998-08-07   13895.0  13895.0  13895.0  13895.0        7.00
1998-08-08   13895.0  13895.0  13895.0  13895.0        0.00

--------------------------------------------------------------------------------

Outlier date: 2011-02-11
            Lần cuối       

In [ ]:
# save cleaned data
# save index as first column
df['Ngày'] = df.index

df.to_csv('../usdvnd/processed/cleaned.csv', index=False)
